In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('rec').getOrCreate()

In [3]:
from pyspark.ml.recommendation import ALS

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
data = spark.read.csv('Dataset/movielens_ratings.csv',inferSchema=True,header=True)

In [6]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [7]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [8]:
training,test=data.randomSplit([0.8,0.2])

In [9]:
als=ALS(maxIter=5,regParam=0.01,userCol='userId',itemCol='movieId',ratingCol='rating')

In [10]:
model=als.fit(training)

In [11]:
predictions=model.transform(test)

In [12]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      1|   1.0|    26| -1.0149581|
|      1|   1.0|     6|  1.7427313|
|      1|   1.0|     7|  1.8511543|
|      3|   1.0|    29|  1.6952223|
|      3|   3.0|    14| -1.2835838|
|      3|   1.0|     0| 0.62254626|
|      5|   2.0|    15| -1.2308211|
|      5|   1.0|     8| -3.5169144|
|      5|   1.0|    29| 0.20842385|
|      5|   2.0|     0|  1.5365778|
|      4|   2.0|    13|  1.4460329|
|      4|   1.0|     9|  0.9679872|
|      4|   2.0|     8| -1.4307852|
|      4|   3.0|    10|-0.83137715|
|      4|   3.0|     2| -2.7730122|
|      4|   3.0|    18| 0.44820815|
|      2|   1.0|    26| -2.2076368|
|      2|   1.0|    23| -1.9150021|
|      0|   1.0|    27|  1.2226675|
|      0|   1.0|     6|  2.9109325|
+-------+------+------+-----------+
only showing top 20 rows



In [13]:
evaluator=RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')

In [14]:
rsme = evaluator.evaluate(predictions)

In [15]:
print('RMSE')
print(rsme)

RMSE
2.161248243281071


In [16]:
single_user=test.filter(test['userId']==11).select(['movieId','userId'])

In [17]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      9|    11|
|     11|    11|
|     27|    11|
|     32|    11|
|     41|    11|
|     45|    11|
|     50|    11|
|     51|    11|
|     59|    11|
|     76|    11|
|     77|    11|
|     97|    11|
+-------+------+



In [18]:
recommendations = model.transform(single_user)

In [19]:
recommendations.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     51|    11|  4.8463407|
|     50|    11|  3.5812318|
|      9|    11|  2.4282513|
|     32|    11|  2.2940688|
|     77|    11|  1.6288851|
|     27|    11|  1.4459703|
|     45|    11| 0.78196526|
|     97|    11| 0.72796005|
|     11|    11|  0.6378095|
|     41|    11| -0.8763012|
|     76|    11|-0.98145074|
|     59|    11| -1.0051414|
+-------+------+-----------+

